In [ ]:
# imports
import csv
import pysam
from pathlib import Path

In [ ]:
def get_bam_stats(bamfile):
    """Read alignment stats from BAM file"""
    bam_stats = {}
    bam_stats["all_primary_alignments"] = int(pysam.view("-c", "-F", "256", bamfile))
    bam_stats["all_mapped_pairs"] = int(pysam.view("-c", "-F", "256", "-F", "8", bamfile))
    bam_stats["proper_pairs"] = int(pysam.view("-c", "-f", "2", bamfile))
    bam_stats["proper_pairs_unique"] = int(pysam.view("-c", "-f", "2", "-q", "1", bamfile))
    bam_stats["proper_pairs_unique_sense"] = int(pysam.view("-c", "-f", "2", "-q", "1", "-f", "64", "-F", "16", bamfile)) * 2
    bam_stats["proper_pairs_unique_antisense"] = int(pysam.view("-c", "-f", "2", "-q", "1", "-f", "64", "-f", "16", bamfile)) * 2
    return(bam_stats)

In [ ]:
def get_human_rna_total(sample):
    """Read human gene counts and report sum for the sample"""
    sample_filenames = list(filter(lambda x: sample in x, snakemake.input.human_small_rna))
    assert len(sample_filenames), 1
    gene_count_filename = sample_filenames[0]
    count = 0
    with open(gene_count_filename) as file:
        tsv_file = csv.reader(file, delimiter="\t")
        for line in tsv_file:
            count = count + int(line[1])
    return(count)

In [ ]:
stats = {}
with open(snakemake.output[0], "w") as output:
    header_line = "sample\talignment category\treads (individual alignments)\tfragments (read pairs)\thuman small rna fragments"
    output.write(f"{header_line}\n")
    print(header_line)
    for filename in (snakemake.input.bam):
        sample_unit = Path(filename).stem
        stats[filename] = get_bam_stats(filename)
        for stat, read_count in stats[filename].items():
            if stat == "all_primary_alignments":
                fragment_count = None
            else:
                fragment_count = int(read_count / 2)
            human_small_rna_count = get_human_rna_total(sample_unit)
            output_line = f"{sample_unit}\t{stat}\t{read_count}\t{fragment_count}\t{human_small_rna_count}"
            output.write(f"{output_line}\n")
            print(output_line)